In [1]:
import pandas as pd
import kagglehub
import numpy as np
import os
import duckdb

import warnings
warnings.filterwarnings("ignore")

/Users/dan/anaconda3/envs/finbert/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset  
df1 = pd.read_csv('../data/bitcoin_sentiments_21_24.csv')
df1

,Date,Short Description,Accurate Sentiments
0,2021-11-05 04:42:00,Bitcoin price is consolidating near the USD 62...,0.998558
1,2021-11-05 08:15:00,Congress could finally approve or reject the m...,0.000000
2,2021-11-05 10:24:00,Bitcoin increasingly becoming a political inst...,0.000000
3,2021-11-05 16:58:00,There is still potential for the price of bitc...,0.999458
4,2021-11-05 21:00:00,'Several companies' are looking to Latin Ameri...,0.000000
...,...,...,...
11290,2024-09-12 00:00:00,The government of El Salvador has been buying ...,0.000000
11291,2024-09-12 00:00:00,"According to data from mempool.space, transact...",0.997017
11292,2024-09-12 00:00:00,Time Magazine reporter Vera Bergengruen believ...,0.000000
11293,2024-09-12 00:00:00,"in bitcoin is concentrated at around $58,500,...",0.000000


In [3]:
# Load the currency dataset
currency = pd.read_csv('../data/CryptoNewsDataset_csvOutput/currency.csv')

# Create a dictionary mapping lowercase names to their corresponding codes
currency_name_to_code = {name.lower(): code for name, code in zip(currency['name'], currency['code'])}

# Create a set of uppercase tickers for direct matching
currency_codes = set(currency['code'])

# Add a new column 'currencies' to df1 based on matches
def find_currencies(description):
    description_str = str(description)
    matched_currencies = set()
    
    # Check for direct matches with tickers (must match uppercase)
    for code in currency_codes:
        if code in description_str:  # Match uppercase tickers
            matched_currencies.add(code)  # Add the ticker as is (uppercase)
    
    # Check for matches with names (case-insensitive) and map to tickers
    description_lower = description_str.lower()
    for name, code in currency_name_to_code.items():
        if name in description_lower:  # Match full names case-insensitively
            matched_currencies.add(code)  # Add the corresponding ticker (uppercase)
    
    return ', '.join(matched_currencies)

df1['currencies'] = df1['Short Description'].apply(find_currencies)
df1

,Date,Short Description,Accurate Sentiments,currencies
0,2021-11-05 04:42:00,Bitcoin price is consolidating near the USD 62...,0.998558,"BTC, XRP, ETH"
1,2021-11-05 08:15:00,Congress could finally approve or reject the m...,0.000000,
2,2021-11-05 10:24:00,Bitcoin increasingly becoming a political inst...,0.000000,"BTC, MKR"
3,2021-11-05 16:58:00,There is still potential for the price of bitc...,0.999458,"BTC, ARK, HI, ETH, T"
4,2021-11-05 21:00:00,'Several companies' are looking to Latin Ameri...,0.000000,BTC
...,...,...,...,...
11290,2024-09-12 00:00:00,The government of El Salvador has been buying ...,0.000000,"BTC, T, DAI"
11291,2024-09-12 00:00:00,"According to data from mempool.space, transact...",0.997017,"BTC, DAI, ORDI, REN, T"
11292,2024-09-12 00:00:00,Time Magazine reporter Vera Bergengruen believ...,0.000000,"BTC, T, DENT"
11293,2024-09-12 00:00:00,"in bitcoin is concentrated at around $58,500,...",0.000000,"HI, BTC, T, ORDI"


In [4]:
# Load the CSV file into a DataFrame
df = pd.read_csv('../data/CryptoNewsDataset_csvOutput/news_currencies_source_joinedResult.csv')

# Create a DuckDB connection
conn = duckdb.connect(database=':memory:')

# Create a table from the DataFrame
conn.execute("CREATE OR REPLACE TABLE news AS SELECT * FROM df")

df2 = conn.execute("SELECT * FROM news").fetchdf()
df2

,id,title,sourceId,source domain,newsDatetime,url,negative,positive,important,liked,disliked,lol,toxic,saved,comments,currencies
0,873,ApeCoin Community Votes to Stay on Ethereum,2,u.today,2022-06-09 06:57:15,https://cryptopanic.com/news/15444990/ApeCoin-...,0,0,0,0,0,0,0,0,0,"ETH,APE"
1,875,Terraform Labs Co-Founder Denies Accusations o...,2,u.today,2022-06-09 06:03:33,https://cryptopanic.com/news/15443520/Terrafor...,0,0,0,1,0,0,0,0,0,None
2,876,Anchor dev claims he warned Do Kwon over unsus...,7,cointelegraph.com,2022-06-09 05:39:22,https://cryptopanic.com/news/15442676/Anchor-d...,1,0,1,0,0,0,0,0,0,LUNC
3,877,"With 12 Key Measures of Prices, Truflation’s R...",11,newsbtc.com,2022-06-09 05:32:52,https://cryptopanic.com/news/15442724/With-12-...,0,0,0,0,0,0,0,0,0,XCN
4,878,'Father' of the Metaverse Neal Stephenson laun...,7,cointelegraph.com,2022-06-09 05:28:31,https://cryptopanic.com/news/15442350/Father-o...,0,1,0,2,0,0,0,0,0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173975,183672,"SingularityNET, ASI Alliance Launch New Proto-...",186,coinscreed.com,2024-11-06 16:43:48,https://cryptopanic.com/news/20176484/Singular...,0,0,0,0,0,0,0,0,0,AGIX
173976,183673,"SingularityNET, ASI alliance launch self-learn...",7,cointelegraph.com,2024-11-06 14:15:00,https://cryptopanic.com/news/20175455/Singular...,0,0,0,0,0,0,0,0,0,AGIX
173977,183674,"AI can save humanity, but only if the people c...",7,cointelegraph.com,2024-10-02 10:50:00,https://cryptopanic.com/news/19961591/AI-can-s...,0,0,0,0,0,0,0,1,0,AGIX
173978,183675,Boba Network and Thrive Protocol Launch Thrive...,12,cryptoglobe.com,2024-10-17 18:38:15,https://cryptopanic.com/news/20038003/Boba-Net...,0,0,0,0,0,0,0,0,0,BOBA


In [5]:
df2.shape

(173980, 16)

In [6]:
# Ensure datetime columns are in datetime format
df1['datetime'] = pd.to_datetime(df1['Date'])
df2['newsDatetime'] = pd.to_datetime(df2['newsDatetime'])

# SQL query to join and create the 'news' column
query = """
SELECT 
    COALESCE(df2.newsDatetime, df1.datetime) AS datetime,
    COALESCE(df2.title, df1."Short Description") AS news,
    COALESCE(df2.currencies, df1.currencies) AS currencies,
    df2.*,
    df1.*
FROM df1
FULL OUTER JOIN df2
ON df1.datetime = df2.newsDatetime
"""

df_merged = conn.execute(query).fetchdf()

df_merged

,datetime,news,currencies,id,title,sourceId,source domain,newsDatetime,url,negative,...,lol,toxic,saved,comments,currencies_1,Date,Short Description,Accurate Sentiments,currencies_2,datetime_1
0,2022-06-09 06:57:15,ApeCoin Community Votes to Stay on Ethereum,"ETH,APE",873,ApeCoin Community Votes to Stay on Ethereum,2,u.today,2022-06-09 06:57:15,https://cryptopanic.com/news/15444990/ApeCoin-...,0,...,0,0,0,0,"ETH,APE",None,None,NaN,None,NaT
1,2022-06-09 06:03:33,Terraform Labs Co-Founder Denies Accusations o...,None,875,Terraform Labs Co-Founder Denies Accusations o...,2,u.today,2022-06-09 06:03:33,https://cryptopanic.com/news/15443520/Terrafor...,0,...,0,0,0,0,None,None,None,NaN,None,NaT
2,2022-06-09 05:39:22,Anchor dev claims he warned Do Kwon over unsus...,LUNC,876,Anchor dev claims he warned Do Kwon over unsus...,7,cointelegraph.com,2022-06-09 05:39:22,https://cryptopanic.com/news/15442676/Anchor-d...,1,...,0,0,0,0,LUNC,None,None,NaN,None,NaT
3,2022-06-09 05:32:52,"With 12 Key Measures of Prices, Truflation’s R...",XCN,877,"With 12 Key Measures of Prices, Truflation’s R...",11,newsbtc.com,2022-06-09 05:32:52,https://cryptopanic.com/news/15442724/With-12-...,0,...,0,0,0,0,XCN,None,None,NaN,None,NaT
4,2022-06-09 05:28:31,'Father' of the Metaverse Neal Stephenson laun...,None,878,'Father' of the Metaverse Neal Stephenson laun...,7,cointelegraph.com,2022-06-09 05:28:31,https://cryptopanic.com/news/15442350/Father-o...,0,...,0,0,0,0,None,None,None,NaN,None,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185127,2024-09-04 00:00:00,The MSTR-BTC sentiment data showed an upward s...,"BTC, T, ARK",<NA>,None,<NA>,None,NaT,None,<NA>,...,<NA>,<NA>,<NA>,<NA>,None,2024-09-04 00:00:00,The MSTR-BTC sentiment data showed an upward s...,0.999541,"BTC, T, ARK",2024-09-04
185128,2024-09-04 00:00:00,Zurich Cantonal Bank has launched Bitcoin and ...,"BTC, ARK, HI, REN, T",<NA>,None,<NA>,None,NaT,None,<NA>,...,<NA>,<NA>,<NA>,<NA>,None,2024-09-04 00:00:00,Zurich Cantonal Bank has launched Bitcoin and ...,0.764590,"BTC, ARK, HI, REN, T",2024-09-04
185129,2024-09-04 00:00:00,Grayscale Research believes that a weakening d...,"BTC, XEM, ARK, REN, T",<NA>,None,<NA>,None,NaT,None,<NA>,...,<NA>,<NA>,<NA>,<NA>,None,2024-09-04 00:00:00,Grayscale Research believes that a weakening d...,-0.814791,"BTC, XEM, ARK, REN, T",2024-09-04
185130,2024-09-04 00:00:00,Cantor Fitzgerald CEO Howard Lutnick has said ...,"USDT, BTC, T, REN",<NA>,None,<NA>,None,NaT,None,<NA>,...,<NA>,<NA>,<NA>,<NA>,None,2024-09-04 00:00:00,Cantor Fitzgerald CEO Howard Lutnick has said ...,-0.885278,"USDT, BTC, T, REN",2024-09-04
